In [3]:
import pandas as pd
import numpy as np
import pickle

# Set seed
random_seed = 123
np.random.seed(random_seed)

# load Preprocessed data from Step 2
with open('preprocessed_data.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract what we need
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']
contamination_rate = data['contamination_rate']

print("✓ Preprocessed data loaded successfully!")
print(f"\nTraining samples: {X_train.shape[0]:,}")
print(f"Test samples: {X_test.shape[0]:,}")
print(f"Features: {X_train.shape[1]}")
print(f"Contamination rate: {contamination_rate:.4f}")

✓ Preprocessed data loaded successfully!

Training samples: 44,394
Test samples: 44,394
Features: 150
Contamination rate: 0.1066
